-What about electrons? (I don't see any - 2 phys.me)

-1e11 and mDM?

-Compute T(E,z_in, z_dep) by doing a delta function injection

# Full Transfer Functions

In [1]:
%load_ext autoreload
import sys
sys.path.append("..")

In [2]:
%autoreload

import matplotlib
matplotlib.rc_file('matplotlibrc')
import matplotlib.pyplot as plt

%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10,10]


from astropy.io import fits

import numpy as np
import copy
import pickle

import darkhistory.physics as phys
import darkhistory.utilities as utils
import darkhistory.spec.spectools as spectools
import darkhistory.spec.transferfunction as tf
import darkhistory.spec.transferfunclist as tflist
from darkhistory.spec.spectrum import Spectrum
from darkhistory.spec.spectra import Spectra
import darkhistory.history.tla as tla

from scipy.interpolate import interp1d

from tqdm import tqdm_notebook as tqdm

from darkhistory.electrons.ics.ics_spectrum import ics_spec
from darkhistory.electrons.ics.ics_spectrum import nonrel_spec
from darkhistory.electrons.ics.ics_spectrum import rel_spec
from darkhistory.electrons.ics.ics_engloss_spectrum import engloss_spec
from darkhistory.electrons.ics.ics_cooling import get_ics_cooling_tf

from darkhistory.electrons import positronium as pos

from darkhistory.low_energy.lowE_deposition import compute_fs

# np.set_printoptions(threshold=np.nan)

In [3]:
soln = pickle.load(open("../darkhistory/history/std_soln.p", "rb"))
xe_std  = interp1d(soln[0,:], soln[2,:])
T_m_std = interp1d(soln[0,:], soln[1,:])

## Photon Transfer Functions

In [4]:
user = 'hongwan'

highengphot_tflist_arr = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_photspec_60eV_complete_coarse.raw", "rb"))
lowengphot_tflist_arr  = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_lowengphotspec_60eV_complete_coarse.raw", "rb"))
lowengelec_tflist_arr  = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_lowengelecspec_60eV_complete_coarse.raw", "rb"))
CMB_engloss_arr = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/CMB_engloss_60eV_complete_coarse.raw", "rb"))

In [5]:
photeng = highengphot_tflist_arr[0].eng
eleceng = lowengelec_tflist_arr[0].eng

#Split photeng into high and low energy. 
photeng_high = photeng[photeng > 60]
photeng_low  = photeng[photeng <= 60]

# Split eleceng into high and low energy. 
eleceng_high = eleceng[eleceng > 3000]
eleceng_low  = eleceng[eleceng <= 3000]


for highengphot_tflist in highengphot_tflist_arr:
    
#     highengphot_tflist.at_val('in_eng', photeng_high, bounds_error=False, fill_value=0)

    for tf in highengphot_tflist:
        # tf.rebin(photeng_high)
#         tf.rebin(photeng)
        
        # Pad with zeros so that it becomes photeng x photeng. 
        tf._grid_vals = np.pad(tf.grid_vals, ((photeng_low.size, 0), (0, 0)), 'constant')
        tf._N_underflow = np.pad(tf._N_underflow, (photeng_low.size, 0), 'constant')
        tf._eng_underflow = np.pad(tf._eng_underflow, (photeng_low.size, 0), 'constant')
        tf._in_eng = photeng
        tf._eng = photeng
        tf._rs = tf.rs[0]*np.ones_like(photeng)
    
    highengphot_tflist._eng = photeng
    highengphot_tflist._in_eng = photeng
    highengphot_tflist._grid_vals = np.atleast_3d(
        np.stack([tf.grid_vals for tf in highengphot_tflist._tflist])
    )
    
# lowengphot_tflist.in_eng set to photeng_high
for lowengphot_tflist in lowengphot_tflist_arr:
    
#     lowengphot_tflist.at_val('in_eng', photeng_high, bounds_error=False, fill_value=0)
    
    for tf in lowengphot_tflist:
#         tf.rebin(photeng)
        # Pad with zeros so that it becomes photeng x photeng. 
        tf._grid_vals = np.pad(tf.grid_vals, ((photeng_low.size,0), (0,0)), 'constant')
        # Photons in the low energy bins should be immediately deposited.
        tf._grid_vals[0:photeng_low.size, 0:photeng_low.size] = np.identity(photeng_low.size)
        tf._N_underflow = np.pad(tf._N_underflow, (photeng_low.size, 0), 'constant')
        tf._eng_underflow = np.pad(tf._eng_underflow, (photeng_low.size, 0), 'constant')
        tf._in_eng = photeng
        tf._eng = photeng
        tf._rs = tf.rs[0]*np.ones_like(photeng)
    
    lowengphot_tflist._eng = photeng
    lowengphot_tflist._in_eng = photeng
    lowengphot_tflist._grid_vals = np.atleast_3d(
        np.stack([tf.grid_vals for tf in lowengphot_tflist._tflist])
    )
    
# lowengelec_tflist.in_eng set to photeng_high 
for lowengelec_tflist in lowengelec_tflist_arr:
    
#     lowengelec_tflist.at_val('in_eng', photeng_high, bounds_error=False, fill_value=0)
    
    for tf in lowengelec_tflist:
#         tf.rebin(eleceng)
        # Pad with zeros so that it becomes photeng x eleceng. 
        tf._grid_vals = np.pad(tf.grid_vals, ((photeng_low.size,0), (0,0)), 'constant')
        tf._N_underflow = np.pad(tf._N_underflow, (photeng_low.size, 0), 'constant')
        tf._eng_underflow = np.pad(tf._eng_underflow, (photeng_low.size, 0), 'constant')
        tf._in_eng = photeng
        tf._eng = eleceng
        tf._rs = tf.rs[0]*np.ones_like(photeng)

    lowengelec_tflist._eng = eleceng
    lowengelec_tflist._in_eng = photeng
    lowengelec_tflist._grid_vals = np.atleast_3d(
        np.stack([tf.grid_vals for tf in lowengelec_tflist._tflist])
    )
    
    for engloss in CMB_engloss_arr:
        engloss = np.pad(engloss, ((0,0),(photeng_low.size, 0)), 'constant')


In [6]:
xes = 0.5 + 0.5*np.tanh([-5., -4.1, -3.2, -2.3, -1.4, -0.5, 0.4, 1.3, 2.2, 3.1, 4])

highengphot_tf_interp = tflist.TransferFuncInterp(xes, highengphot_tflist_arr)
lowengphot_tf_interp = tflist.TransferFuncInterp(xes, lowengphot_tflist_arr)
lowengelec_tf_interp = tflist.TransferFuncInterp(xes, lowengelec_tflist_arr)


## Inverse Compton Scattering

Set up all of the inverse Compton scattering transfer functions (needs to be hidden away at some point). 

In [7]:
# %%prun

# Emax = 1e20
# Emin = 1e-8
# nEe = 5000
# nEp  = 5000

# dlnEp = np.log(Emax/Emin)/nEp
# lowengEp_rel = Emin*np.exp((np.arange(nEp)+0.5)*dlnEp)        

# dlnEe = np.log(Emax/Emin)/nEe
# lowengEe_rel = Emin*np.exp((np.arange(nEe)+0.5)*dlnEe)

# Emax = 1e10
# Emin = 1e-8
# nEe = 5000
# nEp  = 5000

# dlnEp = np.log(Emax/Emin)/nEp
# lowengEp_nonrel = Emin*np.exp((np.arange(nEp)+0.5)*dlnEp)  

# dlnEe = np.log(Emax/Emin)/nEe
# lowengEe_nonrel = Emin*np.exp((np.arange(nEe)+0.5)*dlnEe)

# print('********* Thomson regime scattered photon spectrum *********')
# ics_thomson_ref_tf = nonrel_spec(lowengEe_nonrel, lowengEp_nonrel, phys.TCMB(400))
# print('********* Relativistic regime scattered photon spectrum *********')
# ics_rel_ref_tf = rel_spec(lowengEe_rel, lowengEp_rel, phys.TCMB(400), inf_upp_bound=True)
# print('********* Thomson regime energy loss spectrum *********')
# engloss_ref_tf = engloss_spec(lowengEe_nonrel, lowengEp_nonrel, phys.TCMB(400), nonrel=True)

In [8]:
# pickle.dump(ics_thomson_ref_tf, open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/ics_thomson_ref_tf.p", "wb"))
# pickle.dump(ics_rel_ref_tf,     open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/ics_rel_ref_tf.p", "wb"))
# pickle.dump(engloss_ref_tf,     open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/engloss_ref_tf.p", "wb"))

In [9]:
ics_thomson_ref_tf_2 = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/ics_thomson_ref_tf.p", "rb"))
ics_rel_ref_tf_2 = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/ics_rel_ref_tf.p", "rb"))
engloss_ref_tf_2 = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/engloss_ref_tf.p", "rb"))

In [16]:
# %%prun
# start_rs = 3000
# rs = start_rs
# while rs > 2500:
#     (ics_sec_phot_tf, ics_sec_elec_tf, continuum_loss) = get_ics_cooling_tf(
#         ics_thomson_ref_tf_2, ics_rel_ref_tf_2, engloss_ref_tf_2,
#         eleceng, photeng, rs, fast=True
#     )
#     rs = np.exp(np.log(rs) - 0.002)
#     print(rs)

2994.005996001995
2988.023968031968
2982.053892161797
2976.095744511174
2970.149501247497
2964.215138585785
2958.29263278858
2952.38196016585
2946.4830970748976
2940.5960199202623
2934.720705153627
2928.8571292737256
2923.0052688262463
2917.1651004037394
2911.336600645524
2905.5197462375927
2899.714513912521
2893.9208804493705
2888.1388226736008
2882.3683174569724
2876.609341717457
2870.8618724191438
2865.1258865721484
2859.4013612325193
2853.6882735021477
2847.9866005286754
2842.2963195054017
2836.6174076711964
2830.9498423104037
2825.293600752755
2819.648660373276
2814.0149985921985
2808.392592874867
2802.781420731652
2797.1814597178563
2791.5926874336296
2786.0150815238744
2780.4486196781604
2774.893279630632
2769.349039159922
2763.8158760890615
2758.29376828539
2752.7826936604692
2747.282630169994
2741.7935558137024
2736.3154486352896
2730.8482867223197
2725.3920482061376
2719.9467112617826
2714.512254107899
2709.0886550066516
2703.675892263636
2698.2739442277943
2692.8827892913273

## Evolution

In [11]:
# mDM = 605662.1322589782*2
# mDM = 10**9.104*2
# mDM = 1.27164e9*2
mDM = 10**8*2
# mDM = 10**6.5*2

phot_spec_init = spectools.rebin_N_arr(np.array([2]), 
                                      np.array([mDM/2]), 
                                      photeng)
phot_spec_init.switch_spec_type()

elec_spec_init = spectools.rebin_N_arr(np.array([2]), 
    np.array([mDM/2 - phys.me]),
    eleceng)

elec_spec_init.switch_spec_type()

In [12]:
print(phot_spec_init.toteng(bound_type = 'eng', bound_arr = np.array([1,1e13]))/mDM)

[1.]


In [19]:
%autoreload
phot_spec_init.rs = 3000
elec_spec_init.rs = 3000
end_rs   = 4
#mDM = 1e8
sigmav = 1e-26
tau = 1e25

def rate_func_swave_N(rs):
    # This is dN/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav)/(2*mDM)

def rate_func_swave_eng(rs):
    # This is dE/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav)

def rate_func_decay_N(rs):
    # This is dN/dV dt
    return phys.inj_rate('decay', rs, tau=tau)/mDM

def rate_func_decay_eng(rs):
    # This is dE/dV dt
    return phys.inj_rate('decay', rs, tau=tau)


def rate_func_swave_N_2(rs):
    # This is dN/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav*1e-30)/(2*mDM)

def rate_func_swave_eng_2(rs):
    # This is dE/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav*1e-30)


def evolve(
    in_spec_elec, in_spec_phot, 
    rate_func_N, rate_func_eng, xe_init, T_m_init,
    highengphot_tf_interp, lowengphot_tf_interp, 
    lowengelec_tf_interp, 
    end_rs, coarsen_factor=1, std_soln=False
):
    """
    Main function that computes the temperature and ionization history. 
    
    Parameters
    ----------
    in_spec_elec : Spectrum
        Spectrum per annihilation/decay into electrons. 
    in_spec_phot : Spectrum
        Spectrum per annihilation/decay into photons.
    rate_func_N : function
        Function describing the rate of annihilation/decay, dN/(dV dt)
    rate_func_eng : function
        Function describing the rate of annihilation/decay, dE/(dV dt)
    xe_init : float
        xe at the initial redshift. 
    T_m_init : float
        Matter temperature at the initial redshift. 
    highengphot_tf_interp : TransferFuncInterp
        Interpolation table for high energy photons. 
    lowengphot_tf_interp : TransferFuncInterp
        Interpolation table for low energy photons. 
    lowengelec_tf_interp : TransferFuncInterp
        Interpolation table for low energy electrons. 
    end_rs : float
        Final redshift to evolve to. 
    coarsen_factor : int
        Coarsening to apply to the transfer function matrix. 
    std_soln : bool
        If true, uses the standard TLA solution for f(z). 
    """
    
    # Initialize the next spectrum as None.
    next_highengphot_spec = None
    next_lowengphot_spec  = None
    next_lowengelec_spec  = None
    
    if (
        highengphot_tf_interp.dlnz    != lowengphot_tf_interp.dlnz
        or highengphot_tf_interp.dlnz != lowengelec_tf_interp.dlnz
        or lowengphot_tf_interp.dlnz  != lowengelec_tf_interp.dlnz
    ):
        raise TypeError('TransferFuncInterp objects must all have the same dlnz.')
    
    if in_spec_elec.rs != in_spec_phot.rs:
        raise TypeError('Input spectra must have the same rs.')
    
    # redshift/timestep related quantities. 
    dlnz = highengphot_tf_interp.dlnz
    prev_rs = None
    rs = in_spec_phot.rs
    dt = dlnz/phys.hubble(rs)
    
    # Function that changes the normalization from per annihilation to per baryon. 
    def norm_fac(rs): 
        return rate_func_N(rs) * dt / (phys.nB * rs**3)
    
    # ICS for in_spec_elec goes here.
    
    (ics_sec_phot_tf, ics_sec_elec_tf, continuum_loss) = get_ics_cooling_tf(
        ics_thomson_ref_tf_2, ics_rel_ref_tf_2, engloss_ref_tf_2,
        eleceng, photeng, rs, fast=True
    )
    
    ics_phot_spec = ics_sec_phot_tf.sum_specs(in_spec_elec) * norm_fac(rs)
    ics_lowengelec_spec = ics_sec_elec_tf.sum_specs(in_spec_elec) * norm_fac(rs)
    positronium_phot_spec = pos.weighted_photon_spec(photeng)*in_spec_elec.totN()/2*norm_fac(rs)
    if positronium_phot_spec.spec_type != 'N':
        positronium_phot_spec.switch_spec_type()
        
    print('input electron energy: ', (phys.me*2 + in_spec_elec.toteng())*norm_fac(rs))
    print('ICS secondary photon spectrum energy: ', ics_phot_spec.toteng())
    print('Upscattered photon energy: ', np.dot(in_spec_elec.N, continuum_loss)*norm_fac(rs))
    print('ICS low energy electron spectrum energy: ', ics_lowengelec_spec.toteng() + phys.me*norm_fac(rs))
    print('Positronium spectrum energy: ', positronium_phot_spec.toteng())
    
    init_inj_spec = in_spec_phot * norm_fac(rs) + ics_phot_spec + positronium_phot_spec
    
    # Initialize the Spectra object that will contain all the 
    # output spectra during the evolution.
    out_highengphot_specs = Spectra([init_inj_spec], spec_type=init_inj_spec.spec_type)
    out_lowengphot_specs  = Spectra([], spec_type=init_inj_spec.spec_type)
    out_lowengelec_specs  = Spectra([], spec_type=init_inj_spec.spec_type)
    
    # Initialize the xe and T array that will store the solutions.
    xe_arr  = np.array([xe_init])
    T_m_arr = np.array([T_m_init])
    
    # Load the standard TLA solution if necessary. 
#     if std_soln:
    if True:
        soln = pickle.load(open("../darkhistory/history/std_soln.p", "rb"))
        xe_std  = interp1d(soln[0,:], soln[2,:])
        T_m_std = interp1d(soln[0,:], soln[1,:])

    # Define these methods for speed.
    append_highengphot_spec = out_highengphot_specs.append
    append_lowengphot_spec  = out_lowengphot_specs.append
    append_lowengelec_spec  = out_lowengelec_specs.append
    
    # For the tqdm bar
    pbar = tqdm(total = (np.log(rs) - np.log(end_rs))/dlnz)
    
    # Loop while we are still at a redshift above end_rs.
    while rs > end_rs:
        
        # Update tqdm bar.
        pbar.update(1)
        
        # If prev_rs exists, calculate xe and T_m. 
        if prev_rs is not None:
#             print('here at least?')
            
            # f_continuum, f_lyman, f_ionH, f_ionHe, f_heat
            # f_raw takes in dE/(dV dt)
            if std_soln:
                f_raw = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_std(rs), 0, 0]), rate_func_eng(rs), dt
                )
#                 print("\n SHOULD BE ONE: ", (next_lowengelec_spec.toteng() + next_lowengphot_spec.toteng())/(
#                 np.sum(f_raw) * rate_func_eng(rs) * dt / phys.nB / rs**3), '\n')   
            else:
                f_raw = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_arr[-1], 0, 0]), rate_func_eng(rs), dt
                )
                
            std_f = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_std(rs), 0, 0]), rate_func_eng(rs), dt
                )
            
            backreaction_f = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_arr[-1], 0, 0]), rate_func_eng(rs), dt
                )
            
            f_raw = np.float64(f_raw)
            
            init_cond = np.array([T_m_arr[-1], xe_arr[-1], 0, 0])

            new_vals = tla.get_history(
                init_cond, f_raw[2], f_raw[1], f_raw[4], 
                rate_func_eng, np.array([prev_rs, rs]),
                reion_switch = False
            )

            T_m_arr = np.append(T_m_arr, new_vals[-1,0])
            xe_arr  = np.append(xe_arr,  new_vals[-1,1])
            
        print('T_m at '+str(rs)+': '+ str(T_m_arr[-1]))
        print('x_e at '+str(rs)+': '+ str(xe_arr[-1]))
        print('Standard x_e at '+str(rs)+': '+str(xe_std(rs)))
        print('Standard T_m at '+str(rs)+': '+str(T_m_std(rs)))
        if prev_rs is not None:
            print('Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat: ', backreaction_f)
            print('Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat: ', std_f)
            
        if std_soln:
            highengphot_tf = highengphot_tf_interp.get_tf(rs, xe_std(rs))
            lowengphot_tf  = lowengphot_tf_interp.get_tf(rs, xe_std(rs))
            lowengelec_tf  = lowengelec_tf_interp.get_tf(rs, xe_std(rs))
        else:
            highengphot_tf = highengphot_tf_interp.get_tf(rs, xe_arr[-1])
            lowengphot_tf  = lowengphot_tf_interp.get_tf(rs, xe_arr[-1])
            lowengelec_tf  = lowengelec_tf_interp.get_tf(rs, xe_arr[-1])
        
#         if coarsen_factor > 1:
#             prop_tf = np.zeros_like(highengphot_tf._grid_vals)
#             for i in np.arange(coarsen_factor):
#                 prop_tf += matrix_power(highengphot_tf._grid_vals, i)
#             lowengphot_tf._grid_vals = np.matmul(prop_tf, lowengphot_tf._grid_vals)
#             lowengelec_tf._grid_vals = np.matmul(prof_tf, lowengelec_tf._grid_vals)
#             highengphot_tf._grid_vals = matrix_power(
#                 highengphot_tf._grid_vals, coarsen_factor
#             )
            
        next_highengphot_spec = highengphot_tf.sum_specs(out_highengphot_specs[-1])
        next_lowengphot_spec  = lowengphot_tf.sum_specs(out_highengphot_specs[-1])
        next_lowengelec_spec  = lowengelec_tf.sum_specs(out_highengphot_specs[-1]) + ics_lowengelec_spec
        
#         print('\n SHOULD ALSO BE ONE: ', 
#               out_highengphot_specs[-1].toteng(bound_type = 'eng', bound_arr = np.array([100,1e13]))/(
#         next_highengphot_spec.toteng(bound_type = 'eng', bound_arr = np.array([100,1e13])) + 
#             next_lowengphot_spec.toteng(bound_type = 'eng', bound_arr = np.array([100,1e13])) + 
#             next_lowengelec_spec.toteng(bound_type = 'eng', bound_arr = np.array([100,1e13]))), '\n')
        
#         print('ENERGIES:')
#         print(out_highengphot_specs[-1].toteng())
#         for i in np.arange(500):
#             print(i, highengphot_tf.in_eng[i], highengphot_tf[i].toteng())
    
#         print('\n SHOULD ALSO BE ONE (JUST HIGH ENERGY): ', 
#               out_highengphot_specs[-1].toteng()/(
#         next_highengphot_spec.toteng()), '\n')
        
#         output_eng = next_highengphot_spec.N*next_highengphot_spec.eng/norm_fac(rs)
#         output_N   = next_highengphot_spec.dNdE/norm_fac(rs)
#         utils.compare_arr([next_highengphot_spec.eng, output_eng])
#         print(output_eng[385:391]/1e9)
#         print(output_N[385:391])
#         print(next_highengphot_spec.eng[385:391])
        
#         utils.compare_arr([out_highengphot_specs[-1].N, next_highengphot_spec.N])
        
        # Re-define existing variables.
        prev_rs = rs
        rs = np.exp(np.log(rs) - 0.002)
        
        dt = dlnz/phys.hubble(rs)
        next_highengphot_spec.rs = rs
        next_lowengphot_spec.rs  = rs
        next_lowengelec_spec.rs  = rs
        
        # Add the next injection spectrum to next_highengphot_spec
    
        (ics_sec_phot_tf, ics_sec_elec_tf, continuum_loss) = get_ics_cooling_tf(
            ics_thomson_ref_tf_2, ics_rel_ref_tf_2, engloss_ref_tf_2,
            eleceng, photeng, rs, fast=True
        )

        ics_phot_spec = ics_sec_phot_tf.sum_specs(in_spec_elec) * norm_fac(rs)
        ics_lowengelec_spec = ics_sec_elec_tf.sum_specs(in_spec_elec) * norm_fac(rs)
        positronium_phot_spec = pos.weighted_photon_spec(photeng)*in_spec_elec.totN()/2 * norm_fac(rs)
        if positronium_phot_spec.spec_type != 'N':
            positronium_phot_spec.switch_spec_type()
    

        next_inj_spec = in_spec_phot * norm_fac(rs) + ics_phot_spec + positronium_phot_spec

        # This keeps the redshift. 
        next_highengphot_spec.N += next_inj_spec.N
        
        append_highengphot_spec(next_highengphot_spec)
        append_lowengphot_spec(next_lowengphot_spec)
        append_lowengelec_spec(next_lowengelec_spec)
        
        print('************************ END OF LOOP **************************')
        
        
    return (
        xe_arr, T_m_arr, 
        out_highengphot_specs, out_lowengphot_specs, out_lowengelec_specs
    )

In [20]:
%%prun
# a = evolve(
#         phot_spec_init*0, phot_spec_init, 
#         rate_func_decay_N, rate_func_decay_eng, .9999, phys.TCMB(phot_spec_init.rs),
#         highengphot_tf_interp, lowengphot_tf_interp, 
#         lowengelec_tf_interp, 
#         end_rs, coarsen_factor=1, std_soln=False
#     )

b_t = evolve(
        elec_spec_init, phot_spec_init*0, 
        rate_func_decay_N, rate_func_decay_eng, xe_std(phot_spec_init.rs), phys.TCMB(phot_spec_init.rs),
        #rate_func_decay_N, rate_func_decay_eng, 2e-4, phys.TCMB(phot_spec_init.rs),
        highengphot_tf_interp, lowengphot_tf_interp, 
        lowengelec_tf_interp, 
        end_rs, coarsen_factor=1, std_soln=True
    )

input electron energy:  3.7837473762910907e-06
ICS secondary photon spectrum energy:  3.804844385662529e-06
Upscattered photon energy:  4.053862142027746e-08
ICS low energy electron spectrum energy:  9.775092040538837e-09
Positronium spectrum energy:  1.9334909215821445e-08


HBox(children=(IntProgress(value=0, max=3310), HTML(value='')))

T_m at 3000: 0.70459598745
x_e at 3000: 0.999999999999
Standard x_e at 3000: 0.999999999999
Standard T_m at 3000: 0.70459598745
************************ END OF LOOP **************************
T_m at 2994.005996001995: 0.7031880248617194
x_e at 2994.005996001995: 0.9999999971482519
Standard x_e at 2994.005996001995: 0.9999999971389522
Standard T_m at 2994.005996001995: 0.7031880248257363
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.00058794142564276, 0.0028663816002841534, 0.013536210492576396, 1.7461516786254058e-16, 0.17461516739196506]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.00058794142564276, 0.0028663816002841534, 0.013536210492576396, 1.7461651376831444e-16, 0.17461516739196506]


../darkhistory/spec/spectrum.py:748: RuntimeWarning: The new abscissa lies below the old one: only bins that lie within the new abscissa will be rebinned, bins above the abscissa will be discarded.
  warnings.warn("The new abscissa lies below the old one: only bins that lie within the new abscissa will be rebinned, bins above the abscissa will be discarded.", RuntimeWarning)
../darkhistory/spec/spectrum.py:662: RuntimeWarning: divide by zero encountered in log
  np.log(bound_arr),


************************ END OF LOOP **************************
T_m at 2988.023968031968: 0.7017830534392253
x_e at 2988.023968031968: 0.9999999970473692
Standard x_e at 2988.023968031968: 0.9999999970348642
Standard T_m at 2988.023968031968: 0.7017830533974218
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.0010999824899119944, 0.004731193769387569, 0.016420592068031356, 2.0124521723594455e-16, 0.201243661669945]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.0010999824899119944, 0.004731193769387569, 0.016420592068031356, 2.0124527908672743e-16, 0.201243661669945]
************************ END OF LOOP **************************
T_m at 2982.053892161797: 0.7003808891484844
x_e at 2982.053892161797: 0.9999999969419705
Standard x_e at 2982.053892161797: 0.9999999969267502
Standard T_m at 2982.053892161797: 0.700380889101701
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.0015081331936391065, 0.006197260369616404, 0.018673926648377765, 2.2341148284840293e-1

...

************************ END OF LOOP **************************
T_m at 4.350834073046606: 0.7879516477264056
x_e at 4.350834073046606: 0.8269482440643757
Standard x_e at 4.350834073046606: 0.00018719209328931717
Standard T_m at 4.350834073046606: 3.589959638444936e-05
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.005073958193577501, 0.0347654486136891, 0.21286421106357406, 6.6137080159885e-06, 0.5632300348710564]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.08036611956198084, 0.20354766124357232, 0.3774626438905623, 0.005434469853089473, 0.14912937190070813]
************************ END OF LOOP **************************
T_m at 4.342141100770446: 0.7893074375642106
x_e at 4.342141100770446: 0.8296851268208587
Standard x_e at 4.342141100770446: 0.0001871690673782724
Standard T_m at 4.342141100770446: 3.575331789531795e-05
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.005067192202935547, 0.034776000110652745, 0.21298417839167882, 5.289339263221748e-0

************************ END OF LOOP **************************
T_m at 4.230700438513006: 0.8069218591307833
x_e at 4.230700438513006: 0.8660452746422247
Standard x_e at 4.230700438513006: 0.0001868738837869309
Standard T_m at 4.230700438513006: 3.3878082223909176e-05
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.005008632457604879, 0.03496602370100567, 0.21459519472398517, 4.803525180014517e-07, 0.5595364034642221]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.07987277512345585, 0.2026696174273459, 0.37744072859070915, 0.005334314299735491, 0.14878929925808937]
************************ END OF LOOP **************************
T_m at 4.222247493398743: 0.8082754453534724
x_e at 4.222247493398743: 0.8689029972490085
Standard x_e at 4.222247493398743: 0.00018685149365889596
Standard T_m at 4.222247493398743: 3.373584272276453e-05
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.005005914287835247, 0.03498297161147897, 0.21472215263634895, 4.110703648121215

************************ END OF LOOP **************************
T_m at 4.113883889830981: 0.8258422321595646
x_e at 4.113883889830981: 0.9068618841897031
Standard x_e at 4.113883889830981: 0.00018656446056860486
Standard T_m at 4.113883889830981: 3.191238537087591e-05
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.004981397128794806, 0.035219647972840905, 0.2162911272178319, 9.768136937759064e-08, 0.5556656806299822]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.07936233539129027, 0.20175600575969782, 0.3773776411226012, 0.005232659503314051, 0.14842930885391592]
************************ END OF LOOP **************************
T_m at 4.105664344336661: 0.8271908780288564
x_e at 4.105664344336661: 0.909844834588372
Standard x_e at 4.105664344336661: 0.00018654268866857252
Standard T_m at 4.105664344336661: 3.177407333466324e-05
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.004977611528077959, 0.0352302084134037, 0.2163831142201302, 9.580921334370351e-0

************************ END OF LOOP **************************
T_m at 4.000292836842708: 0.8446766119353064
x_e at 4.000292836842708: 0.9494609675182971
Standard x_e at 4.000292836842708: 0.00018626358103068644
Standard T_m at 4.000292836842708: 3.0000964618328547e-05
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.0049727386538562105, 0.03548257566116049, 0.21776017970024772, 7.613200859130516e-08, 0.5518902984754089]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.07883705066498263, 0.20081368850306364, 0.3772761644251015, 0.005130074886663283, 0.14804889014287095]
************************ END OF LOOP **************************
 

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/Users/hongwan/anaconda/envs/python3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/hongwan/anaconda/envs/python3/lib/python3.6/site-packages/tqdm/_monitor.py", line 63, in run
    for instance in self.tqdm_cls._instances:
  File "/Users/hongwan/anaconda/envs/python3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



In [15]:
plt.figure()
# plt.plot(a[3].rs, a[0])
plt.plot(b[3].rs, b[0])
ax = plt.gca()
ax.set_xscale('log')
ax.set_yscale('log')


plt.figure()
# plt.plot(a[3].rs, a[1]/phys.kB)
plt.plot(b[3].rs, b[1]/phys.kB)
plt.plot(b[3].rs, phys.TCMB(b[3].rs)/phys.kB, 'k:')
ax = plt.gca()
ax.set_xscale('log')
ax.set_yscale('log')


NameError: name 'b' is not defined

<Figure size 720x720 with 0 Axes>

In [ ]:
test_tf = highengphot_tf_interp.get_tf(4, xe_std(3000))
highengphot_tf_interp.xe

In [ ]:
plt.figure()

ax = plt.gca()

print(test_tf.in_eng[375])

test_tf.plot(ax, ind=250, indtype='ind')
test_tf.plot(ax, ind=377, indtype='ind')


ax.set_xscale('log')
ax.set_yscale('log')
plt.axis([1e-4, 7e12, 1e-42, 1e7])

In [ ]:
test_spectra = highengphot_tf_interp._grid_vals[-1,-1,:,:]

plt.figure()
ax = plt.gca()

plt.plot(highengphot_tf_interp.eng, test_spectra[370, :])

ax.set_xscale('log')
ax.set_yscale('log')
plt.axis([1e-4, 7e12, 1e-42, 1e7])

In [ ]:
tf_before_interp = highengphot_tflist_arr[-1][-1]
tf_before_interp_2 = highengphot_tflist_arr[-1][0]

# print(tf_before_interp.in_eng[376])

plt.figure()
ax = plt.gca()
# tf_before_interp.plot(ax, ind=200, indtype='ind')
tf_before_interp.plot(ax, ind=370, indtype='ind')
# tf_before_interp_2.plot(ax, ind=376, indtype='ind')


ax.set_xscale('log')
ax.set_yscale('log')
plt.axis([1e-4, 7e12, 1e-42, 1e7])

In [ ]:
lowengphot_spectra = b[3]
print(lowengphot_spectra.in_eng)

lowengphot_spectra *= (rate_func_decay_N(3000) * (0.002/phys.hubble(3000)) / (phys.nB * 3000**3))**(-1)

print(lowengphot_spectra[0].toteng())

plt.figure()
ax = plt.gca()

lowengphot_spectra.switch_spec_type()
lowengphot_spectra.plot(ax, ind=0, indtype='ind')
lowengphot_spectra.switch_spec_type()

ax.set_xscale('log')
ax.set_yscale('log')

plt.axis([1e-4, 1e3, 1e-10, 1e6])

In [ ]:
lowengelec_spectra = b[4]

lowengelec_spectra *= (rate_func_decay_N(3000) * (0.002/phys.hubble(3000)) / (phys.nB * 3000**3))**(-1)

print(lowengelec_spectra[0].toteng())
utils.compare_arr([lowengelec_spectra[0].eng, lowengelec_spectra[0].dNdE])

plt.figure()
ax = plt.gca()

lowengelec_spectra.switch_spec_type()
lowengelec_spectra.plot(ax, ind=0, indtype='ind')
lowengelec_spectra.switch_spec_type()

ax.set_xscale('log')
ax.set_yscale('log')

plt.axis([1, 3e3, 1e-4, 1e3])

In [ ]:
highengphot_spectra = b[2]

highengphot_spectra *= (rate_func_decay_N(3000) * (0.002/phys.hubble(3000)) / (phys.nB * 3000**3))**(-1)

print(2*10**9.104)
print(highengphot_spectra[0].toteng())
print(highengphot_spectra[1].toteng())

plt.figure()
ax = plt.gca()

highengphot_spectra.switch_spec_type()
highengphot_spectra.plot(ax, ind=1, indtype='ind')
highengphot_spectra.switch_spec_type()

ax.set_xscale('log')
ax.set_yscale('log')

plt.axis([1e-4,5e12, 1e-40, 1e10])